In [1]:
import html5lib
import requests
import lxml
from bs4 import BeautifulSoup
from bs4 import Comment
import pandas as pd
import numpy as np
from helpers import col_desc
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
pd.set_option('mode.chained_assignment', None)

In [2]:
#Getting the teams acronims
teams = pd.read_csv('mlb_teams_abbreviations.csv')
teams = list(teams.iloc[:,0])

In [38]:
# Get the roster table from the placeholder comment
def is_comment(element): 
    return isinstance(element, Comment)

dfs = []

for t in teams:
    url = 'https://www.baseball-reference.com/teams/'+t+'/2019.shtml'
    html = requests.get(url).content
    soup =  BeautifulSoup(html, 'html5lib')
    try:
        roster = soup.find(attrs={'id':'all_the40man'}).find(text=is_comment)
        man40 = pd.read_html(str(BeautifulSoup(roster).find(attrs={'id':'the40man'})))[0]
    except:
        continue
    #rosters = pd.read_html(str(BeautifulSoup(roster).find(attrs={'id':'the40man'})))[0]
    dfs.append(man40)
rosters = pd.concat(dfs)

In [39]:
rosters

,Rk,Uni,Name,Unnamed: 3,Unnamed: 4,OnActv,IL,Age,B,T,Ht,Wt,DoB,1stYr
0,1,23,Zac Gallen,us US,Pitcher,*,NaN,23,R,R,"6' 2""",191,"Aug 3, 1995",2014
1,2,48,Corbin Martin,us US,Pitcher,*,NaN,23,R,R,"6' 2""",228,"Dec 28, 1995",2015
2,3,50,Yoan López,cu CU,Pitcher,*,NaN,26,R,R,"6' 3""",205,"Jan 2, 1993",2011
3,4,38,Robbie Ray,us US,Pitcher,*,NaN,27,L,L,"6' 2""",215,"Oct 1, 1991",2010
4,5,29,Merrill Kelly,us US,Pitcher,*,NaN,30,R,R,"6' 2""",210,"Oct 14, 1988",2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34,35,65,Elvis Luciano,do DO,Pitcher,NaN,NaN,19,R,R,"6' 3""",200,"Feb 15, 2000",2017
35,36,47,Anthony Kay,us US,Pitcher,NaN,NaN,24,L,L,"6' 0""",218,"Mar 21, 1995",2014
36,37,69,Tom Hatch,us US,Pitcher,NaN,NaN,24,R,R,"6' 1""",200,"Sep 29, 1994",2014
37,38,72,Santiago Espinal,do DO,Position,NaN,NaN,24,R,R,"5' 10""",175,"Nov 13, 1994",2016


In [40]:
#removing duplicate headers
rosters = rosters.drop(rosters[rosters['Rk'] == 'Rk'].index)

In [41]:
#Fixing missing column names, accordin to the website the 4th column correspond to the player country and the 5th supposed to be his role but is not very clear
#Chaging the names to be more clear
rosters.columns = ['Rk', 'number', 'name', 'country', 'role', 'active', 'injured', 'age',
                 'batting_side', 'throwing_hand', 'height', 'weight', 'birt_date', 'first_year']

In [42]:
#Removing Rk (rank) role columns
rosters.drop(['Rk','role'], axis=1, inplace=True)

#Removing the duplicate label on the country column
rosters['country'] = rosters['country'].str.split(expand=True)[1]

#Converting active and injured into binary classification 1 is injured, or active while 0 is not for any of them.
rosters['active'] = np.where(rosters['active'].isna(),0,1)
rosters['injured'] = np.where(rosters['injured'].isna(),0,1)

#Formmating the height. Height will be expressed in inches.
rosters['height'] = rosters['height'].str.split(expand=True)[0].str.replace("'",'').astype(int) * 12 + rosters['height'].str.split(expand=True)[1].str.replace('"','').astype(int)

# Converting weight to numeric
rosters['weight'] = pd.to_numeric(rosters['weight'])

# Convertin birth_date to date time format
rosters['birt_date'] = pd.to_datetime(rosters['birt_date'])

# There are some duplicated rows, I removed them:
rosters.drop_duplicates(inplace=True)

In [ ]:
#########################################################################################################

In [31]:
#Getting batting statistics
dfs = []

for t in teams:
    url = 'https://www.baseball-reference.com/teams/'+t+'/2018.shtml'
    html = requests.get(url).content
    soup =  BeautifulSoup(html, 'html5lib')
    try:
        bat = pd.read_html(str(soup.find(attrs={'id':'team_batting'})))[0]
    except:
        continue
    dfs.append(bat)
batting = pd.concat(dfs)    

In [32]:
#Remove repeating headings
batting = batting.drop(batting[batting['Rk'] == 'Rk'].index)
#Remove the subtotal rows
batting = batting.drop(batting[batting['Rk'].isna()].index)
#Remove duplicated rows
batting.drop_duplicates(inplace=True)

In [102]:
#Joing both tables on names
df =  pd.merge(left=batting, right=rosters, how='inner', left_on='Name', right_on='name')

In [103]:
df = df[['age', 'batting_side', 'height', 'weight', 'first_year','G', 'PA', 'R', 'SB', 'SO', 'country', 'OPS']]

In [104]:
#Remove records with missing OPS
df.dropna(axis=0, inplace=True)

In [106]:
#Convert numerical values into integers
def change_num_type(dtf):    
    for c in list(dtf.columns):
        try:
            dtf[c] = dtf[c].astype(int)
        except:
            pass
change_num_type(df)

df['OPS'] = df['OPS'].astype(float)

In [107]:
df.columns = ['age', 'batting_side', 'height', 'weight', 'first_year', 
              'games_played', 'plates_appearances', 'runs',
               'stolen_bases', 'strike_outs', 'country', 'OPS']

In [136]:
#Splitting the dataset into train and test sets.

from sklearn.model_selection import train_test_split
y = df[['OPS']]
X = df.drop('OPS', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35, random_state=28)
train = pd.concat([X_train,y_train], axis=1)
test = pd.concat([X_test,y_test], axis=1)